In [1]:
# Imports
import os
import json
import logging
import sys
from typing import List

# Assuming your notebook is in a subdirectory like 'analysis' or 'notebooks'
# Adjust the path manipulation if your notebook is located elsewhere relative to the root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) 

# Or, if you know the absolute path is always the same:
# project_root = '/local/home/zhamers/L-SARP' 

if project_root not in sys.path:
    print(f"Adding project root to Python path: {project_root}")
    sys.path.insert(0, project_root)
else:
    print(f"Project root already in Python path: {project_root}")

from utils.recursive_config import Config
from configs.goal_execution_log_models import GoalExecutionLogsCollection

# Configurations and Logging
config = Config()

Adding project root to Python path: /local/home/zhamers/L-SARP


In [2]:
# Get the path for log analysis
path_to_scene_data = config.get('robot_planner_settings').get('path_to_scene_data')
active_scene = config.get('robot_planner_settings').get('active_scene')
logs = ['execution_logs_goals_12_full_run_1_20250405_032700.json', 'execution_logs_goals_12_full_run_2_20250405_034224.json', 'execution_logs_goals_12_full_run_3_20250405_040526.json']
log_paths = [os.path.join(project_root, path_to_scene_data, active_scene, log) for log in logs]

In [3]:
# Let's process the 1st log file as a start
path_for_log_analysis = log_paths[0]

# Load the JSON data from the file
with open(path_for_log_analysis) as f:
    raw_logs_data = json.load(f)

# Extract the log entries (values) and put them in the expected structure
formatted_logs_data = {"goal_execution_logs": list(raw_logs_data.values())}

# Validate the formatted data using the Pydantic model
goal_execution_logs_collection = GoalExecutionLogsCollection.model_validate(formatted_logs_data)

# First Step: Demonstrate the Actual actions that the robot has taken.

For each task, we will showcase:
1. To which plan did the task belong? At a replan moment, we show both the old and the new (first) task in the replanned plan
2. Which tools get executed by the task execution agent for each task

We want to show:
1. A series of all the actions (tool calls) that have happened over the whole framework in chronological order


In [4]:
# Anlylize the executed tasks

# Import the table generator function and pandas
import pandas as pd
# Note the function name change here
from analysis.task_table_generator import generate_task_summary_tables_per_goal

# Generate the list of tables (one per goal)
# The function now returns a list of tuples: (goal_number, dataframe, latex_string)
list_of_tables = generate_task_summary_tables_per_goal(goal_execution_logs_collection)

# Display/print each table
for goal_num, df_goal_summary, latex_goal_summary in list_of_tables:
    # Print a header for the notebook output
    print(f"\n--- Goal {goal_num} Task Execution Summary Table ---")
    display(df_goal_summary)  # Display DataFrame in notebook

    # Print the LaTeX representation for this goal's table
    print(f"\n\nLaTeX Representation for Goal {goal_num}:\n")
    print(latex_goal_summary)
    print("-" * 80) # Separator line for readability



--- Goal 1 Task Execution Summary Table ---


,Task Description,Tool Calls / Responses,Plan ID,Inference Time (s)
0,Calculate the bounding box volume of each TV i...,Text: I will calculate the bounding ...<br>obj...,0,2.632
1,Identify the TV with the largest bounding box ...,"Text: Based on the core findings, th...<br>sto...",0,1.538
2,Navigate to the biggest TV.,"Text: Okay, I need to navigate to th...<br>Rob...",0,1.484
3,Final Goal Completion Check,"The robot has navigated to the biggest TV, so ...",N/A,0.615




LaTeX Representation for Goal 1:

\begin{table}
\caption{Summary of Executed Tasks for Goal 1: Go to the biggest tv.}
\label{tab:task_summary_goal_1}
\begin{tabular}{p{6cm}p{5cm}cc}
\toprule
Task Description & Tool Call / Response & Plan ID & Time (s) \\
\midrule
Calculate the bounding box volume of each TV in the scene. & Text: I will calculate the bounding ...
object\_bounding\_box\_volume(object\_id=1.0)
object\_bounding\_box\_volume(object\_id=12.0)
Text: The bounding box volume of TV ...
store\_core\_information(agent\_name=TaskExecutionAgent, information=The bounding box volume of TV with ID 1 is 0.1188230793917458. The bounding box volume of TV with ID 12 is 0.10098798853225036.)
Final Response: I have calculated the bounding... & 0 & 2.632 \\
Identify the TV with the largest bounding box volume. & Text: Based on the core findings, th...
store\_core\_information(agent\_name=TaskExecutionAgent, information=The TV with the largest bounding box volume is TV with ID 1.)
Final Resp

,Task Description,Tool Calls / Responses,Plan ID,Inference Time (s)
0,Navigate to a recycling bin.,Text: I will navigate to a recycling...<br>Rob...,0,1.037
1,Place the object in the recycling bin.,Text: I need to place the object in ...<br>com...,0,2.861
2,Grasp an object.,Text: I need to grasp an object. Loo...<br>gra...,1,4.486
3,Navigate to a recycling bin.,Final Response: The task is to navigate to a r...,1,0.712
4,Place the object in the recycling bin.,Final Response: The task is to place the objec...,1,0.668
5,Navigate to the bottle.,Text: The task is to navigate to the...<br>Rob...,2,1.325
6,Grasp the bottle.,grasp_object(object_id=19.0)<br>Final Response...,2,1.315
7,Navigate to a recycling bin.,Final Response: The task is to navigate to the...,2,0.789
8,Place the bottle in the recycling bin.,Text: The Goal Checker has completed...<br>pla...,2,4.172
9,Final Goal Completion Check,The current scene graph indicates that the rob...,N/A,1.278




LaTeX Representation for Goal 2:

\begin{table}
\caption{Summary of Executed Tasks for Goal 2: Place an object in the recycling bin.}
\label{tab:task_summary_goal_2}
\begin{tabular}{p{6cm}p{5cm}cc}
\toprule
Task Description & Tool Call / Response & Plan ID & Time (s) \\
\midrule
Navigate to a recycling bin. & Text: I will navigate to a recycling...
RobotNavigation(object\_id=8.0, object\_description=recycling bin)
Final Response: I am now navigating to the rec... & 0 & 1.037 \\
Place the object in the recycling bin. & Text: I need to place the object in ...
communicate\_with\_task\_planner(request=I am at the recycling bin, but I don't have an object to place. I need to grasp an object first.)
Final Response: I have communicated with the t... & 0 & 2.861 \\
Grasp an object. & Text: I need to grasp an object. Loo...
grasp\_object(object\_id=19.0)
Text: I am not close enough to the b...
communicate\_with\_task\_planner(request=I tried to grasp the bottle with ID 19, but I am not close 

...



LaTeX Representation for Goal 10:

\begin{table}
\caption{Summary of Executed Tasks for Goal 10: Navigate to and then press each light switch in the room in order from lowest to highest position.}
\label{tab:task_summary_goal_10}
\begin{tabular}{p{6cm}p{5cm}cc}
\toprule
Task Description & Tool Call / Response & Plan ID & Time (s) \\
\midrule
Identify all light switches in the scene graph. & Text: I have identified the followin...
store\_core\_information(agent\_name=TaskExecutionAgent, information=Identified light switches with IDs 28, 29, 30, and 31.)
Final Response: I have stored the information ... & 0 & 1.600 \\
Calculate the vertical (y-axis) position of each light switch. & Text: I will now calculate the verti...
store\_core\_information(agent\_name=TaskExecutionAgent, information=Calculated y-coordinates of light switches: 28: -1.302, 29: -1.274, 30: -0.113, 31: -1.294)
Final Response: I have stored the y-coordinate... & 0 & 2.422 \\
Sort the light switches based on their vert

,Task Description,Tool Calls / Responses,Plan ID,Inference Time (s)
0,Identify all drawers in the scene graph.,"Text: Okay, I need to identify all t...<br>sto...",0,1.306
1,Calculate the vertical (y-axis) position of ea...,"Text: Okay, I need to calculate the ...<br>sto...",0,2.889
2,Sort the drawers from highest to lowest vertic...,Text: I have the vertical positions ...<br>sor...,0,3.500
3,"Iterate through the sorted list of drawers, na...","Text: Okay, I will iterate through t...<br>Rob...",0,5.025
4,"For each drawer, inspect the objects inside an...",inspect_object_with_gaze(object_id=25.0)<br>Te...,0,8.758
5,Report the drawer and the object that seems ou...,"Text: Okay, I have to report the dra...<br>com...",0,8.812
6,Final Goal Completion Check,"The goal is to inspect all drawers, identify a...",N/A,1.521




LaTeX Representation for Goal 11:

\begin{table}
\caption{Summary of Executed Tasks for Goal 11: Inspect the contents of all drawers in the room, ordered from highest to lowest position. Identify and report which drawer contains an object that seems out of place compared to the others in that drawer.}
\label{tab:task_summary_goal_11}
\begin{tabular}{p{6cm}p{5cm}cc}
\toprule
Task Description & Tool Call / Response & Plan ID & Time (s) \\
\midrule
Identify all drawers in the scene graph. & Text: Okay, I need to identify all t...
store\_core\_information(agent\_name=TaskExecutionAgent, information=The drawers in the scene graph have IDs 25, 26, and 27.)
Final Response: I have identified all the draw... & 0 & 1.306 \\
Calculate the vertical (y-axis) position of each drawer. & Text: Okay, I need to calculate the ...
store\_core\_information(agent\_name=TaskExecutionAgent, information=The vertical positions of the drawers are: Drawer 25: 0.45307798115907993, Drawer 26: 0.45846588506453206,

,Task Description,Tool Calls / Responses,Plan ID,Inference Time (s)
0,Navigate to the potted plant to examine it.,"RobotNavigation(object_id=24.0, object_descrip...",0,1.120
1,Inspect the plant with gaze.,Text: The Goal Checker has completed...<br>ins...,0,5.636
2,Final Goal Completion Check,goal completed,N/A,0.420




LaTeX Representation for Goal 12:

\begin{table}
\caption{Summary of Executed Tasks for Goal 12: Examine the plant to see if it needs water. Please water it when it needs water.}
\label{tab:task_summary_goal_12}
\begin{tabular}{p{6cm}p{5cm}cc}
\toprule
Task Description & Tool Call / Response & Plan ID & Time (s) \\
\midrule
Navigate to the potted plant to examine it. & RobotNavigation(object\_id=24.0, object\_description=potted plant)
Final Response: OK. I am moving to the potted ... & 0 & 1.120 \\
Inspect the plant with gaze. & Text: The Goal Checker has completed...
inspect\_object\_with\_gaze(object\_id=24.0)
Text: OK. I have inspected the potte...
store\_core\_information(agent\_name=TaskExecutionAgent, information=The potted plant contains artificial flowers and does not need water.)
Text: OK. I have stored the informat...
check\_if\_goal\_is\_completed(explanation=The task was to examine the plant and water if it needs water. The plant has been examined and it contains artifici

In [5]:
# Success Rate (self proclaimed)
amount_of_goals = len(goal_execution_logs_collection.goal_execution_logs)
amount_of_goals_completed = sum(log.goal_completed for log in goal_execution_logs_collection.goal_execution_logs)
success_rate = amount_of_goals_completed / amount_of_goals
print(f"Self proclaimed Success Rate: {success_rate}")



Self proclaimed Success Rate: 1.0
